<a href="https://colab.research.google.com/github/tusharlawande/Python-Doveloper-Intern/blob/main/Python_Doveloper_Intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required Libraries
!pip install pandas requests backtrader pyarrow

import pandas as pd
import requests
import backtrader as bt
import os
import zipfile
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
DATA_URL = "https://drive.google.com/uc?id=1lsyyilFMvTNGLJlGbqrxVBQVFaesnT93"  # Adjusted URL for direct download
DATA_DIR = "historical_data"  # Directory to store data

# Step 1: Data Handling Plan
def download_and_extract_data(url, extract_to):
    """Download and extract data from the specified URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses

        zip_file_path = os.path.join(extract_to, "data.zip")

        # Write the zip file
        with open(zip_file_path, 'wb') as f:
            f.write(response.content)

        # Extract the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)

        logging.info(f"Data downloaded and extracted to {extract_to}")

    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to download data: {e}")
    except zipfile.BadZipFile:
        logging.error("Failed to extract data: Bad zip file")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

def load_data(derivative, expiry_date):
    """Load and filter data based on derivative and expiry date."""
    data_files = [file for file in os.listdir(DATA_DIR) if file.endswith('.feather')]

    data = []
    for file in data_files:
        df = pd.read_feather(os.path.join(DATA_DIR, file))
        # Filtering for the specific derivative and expiry date
        df_filtered = df[(df['derivative'] == derivative) & (df['expiry_date'] == expiry_date)]
        data.append(df_filtered)

    if data:
        return pd.concat(data).reset_index(drop=True)
    else:
        logging.warning("No data found for the specified derivative and expiry.")
        return None

# Step 2: User Input Handling
def get_user_inputs():
    """Get user inputs for derivative type, timeframes, expiry date, and strategy."""
    derivative = input("Choose derivative (Nifty, BankNifty, FinNifty, BankEx): ")
    expiry_date = input("Enter expiry date (YYYY-MM-DD): ")
    timeframes = input("Choose timeframes for backtest (comma-separated, e.g., 1m, 5m, 1h, 1d): ")
    strategy = input("Choose strategy (MA_Crossover, RSI): ")
    return derivative, expiry_date, timeframes.split(','), strategy

# Step 3: Strategy Implementation
class MovingAverageCrossover(bt.Strategy):
    params = (('short_period', 10), ('long_period', 30), ('stop_loss', 0.02))

    def __init__(self):
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)
        self.order = None  # Track ongoing orders

    def next(self):
        if self.order:
            return  # If an order is pending, do nothing

        if not self.position:  # No position open
            if self.short_ma > self.long_ma:
                self.order = self.buy()  # Buy signal

        elif self.position:  # Position is open
            if self.short_ma < self.long_ma:
                self.order = self.sell()  # Sell signal

    def stop(self):
        logging.info(f'Ending Value: {self.broker.getvalue()}')

class RSI(bt.Strategy):
    params = (('rsi_period', 14), ('overbought', 70), ('oversold', 30))

    def __init__(self):
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_period)
        self.order = None  # Track ongoing orders

    def next(self):
        if self.order:
            return  # If an order is pending, do nothing

        if not self.position:  # No position open
            if self.rsi < self.params.oversold:
                self.order = self.buy()  # Buy signal

        elif self.position:  # Position is open
            if self.rsi > self.params.overbought:
                self.order = self.sell()  # Sell signal

    def stop(self):
        logging.info(f'Ending Value: {self.broker.getvalue()}')

# Step 4: Backtesting
def backtest_strategy(data, strategy_class, timeframe):
    """Backtest the chosen strategy on the provided data."""
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy_class)

    # Create a data feed based on the chosen timeframe
    data_feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_feed)
    cerebro.broker.set_cash(100000)
    cerebro.run()
    cerebro.plot()

def main():
    # Create data directory if it doesn't exist
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    # Download and extract data
    download_and_extract_data(DATA_URL, DATA_DIR)

    # Get user inputs
    derivative, expiry_date, timeframes, strategy = get_user_inputs()

    # Load data
    data = load_data(derivative, expiry_date)

    if data is not None:
        for timeframe in timeframes:
            try:
                # Ensure index is datetime and resample data according to the chosen timeframe
                data['timestamp'] = pd.to_datetime(data['timestamp'])
                data.set_index('timestamp', inplace=True)

                resampled_data = data.resample(timeframe).agg({
                    'open': 'first',
                    'high': 'max',
                    'low': 'min',
                    'close': 'last',
                    'volume': 'sum'
                })
                resampled_data.dropna(inplace=True)

                # Perform backtest based on the strategy chosen
                if strategy == "MA_Crossover":
                    backtest_strategy(resampled_data, MovingAverageCrossover, timeframe)
                elif strategy == "RSI":
                    backtest_strategy(resampled_data, RSI, timeframe)
                else:
                    logging.error("Invalid strategy. Please choose either MA_Crossover or RSI.")
            except Exception as e:
                logging.error(f"Error processing timeframe {timeframe}: {e}")

if __name__ == "__main__":
    main()


ERROR:root:Failed to extract data: Bad zip file


Choose derivative (Nifty, BankNifty, FinNifty, BankEx): Nifty 
